This is just checking the the data info 

In [2]:
import pandas as pd
csv_path = '/Users/hanfuhou/Downloads/2004to2024all.csv' 
df = pd.read_csv(csv_path)
print("Column Names:")
print(df.columns.tolist())
print("\n Data Types:")
print(df.dtypes)

print("\n  Sample Data (first 5 rows):")
print(df.head())

print("\n  Basic Statistics (numeric columns):")
print(df.describe())

print("\n DataFrame Info:")
df.info()

/var/folders/x7/dd18rcl92fl22lg10gbjt8zm0000gn/T/ipykernel_29297/18498857.py:3: DtypeWarning: Columns (6,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Column Names:
['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'type', 'active_lock_reason', 'sub_issues_summary', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'score']

 Data Types:
url                          object
repository_url               object
labels_url                   object
comments_url                 object
events_url                   object
html_url                     object
id                           object
node_id                      object
number                        int64
title                        object
user                         object
labels                       object
state                        object
locked                       object
assignee                     object
assignees             

In [3]:
import pandas as pd


csv_path = '/Users/hanfuhou/Downloads/2004to2024all.csv' 
df = pd.read_csv(csv_path)

# Step 1: Total issues
total_issues = len(df)

# Step 2: Accessibility mentions in labels or title or body
df['labels'] = df['labels'].astype(str).str.lower()
df['title'] = df['title'].astype(str).str.lower()
df['body'] = df['body'].astype(str).str.lower()
accessibility_issues = df[
    df['labels'].str.contains('accessibility|a11y|screen reader|contrast|aria|colorblind|blind|deaf|hearing|ally', na=False) |
    df['title'].str.contains('accessibility|a11y|screen reader|contrast|aria|colorblind|blind|deaf|hearing|ally', na=False) |
    df['body'].str.contains('accessibility|a11y|screen reader|contrast|aria|colorblind|blind|deaf|hearing|ally', na=False)
]
accessibility_count = len(accessibility_issues)
accessibility_ratio = accessibility_count / total_issues

# Step 3: Comments statistics
df['comments_count'] = pd.to_numeric(df['comments'], errors='coerce')
comments_mean = df['comments_count'].mean()
comments_median = df['comments_count'].median()

# Step 4: Author association distribution
author_dist = df['author_association'].value_counts(dropna=False)

# Step 5: Issue states and reasons
state_counts = df['state'].value_counts(dropna=False)
state_reason_counts = df['state_reason'].value_counts(dropna=False)

# Step 6: Score basic stats
score_stats = df['score'].describe()

# === Output ===
print(f"Total issues: {total_issues}")
print(f"Accessibility-related issues: {accessibility_count} ({accessibility_ratio:.2%})")
print(f"Average comments per issue: {comments_mean:.2f}")
print(f"Median comments per issue: {comments_median}")
print("\nAuthor Association Distribution:")
print(author_dist)
print("\nIssue State Counts:")
print(state_counts)
print("\nState Reason Counts:")
print(state_reason_counts)
print("\nScore Stats:")
print(score_stats)

/var/folders/x7/dd18rcl92fl22lg10gbjt8zm0000gn/T/ipykernel_29297/2787716020.py:5: DtypeWarning: Columns (6,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


Total issues: 404067
Accessibility-related issues: 349340 (86.46%)
Average comments per issue: 4.29
Median comments per issue: 1.0

Author Association Distribution:
author_association
NONE            204096
CONTRIBUTOR      97710
OWNER            45418
MEMBER           32920
COLLABORATOR     23708
MANNEQUIN          213
NaN                  1
1.0                  1
Name: count, dtype: int64

Issue State Counts:
state
closed         294437
open           109628
NaN                 1
CONTRIBUTOR         1
Name: count, dtype: int64

State Reason Counts:
state_reason
completed      285437
NaN            108016
not_planned      8853
reopened         1614
duplicate         147
Name: count, dtype: int64

Score Stats:
count    404065.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: score, dtype: float64


Author Roles vs. Comments (Issue Engagement)
Average and median comments by author_association
Distribution of comments by author roles
Do OWNER/MEMBER/CONTRIBUTOR issues tend to have more comments?

In [4]:
import pandas as pd
from scipy.stats import spearmanr
import ast

csv_path = '/Users/hanfuhou/Downloads/2004to2024all.csv' 
df = pd.read_csv(csv_path)

# Clean 'author_association'
df['author_association'] = df['author_association'].fillna('NONE')

# Clean 'comments'
df['comments'] = pd.to_numeric(df['comments'], errors='coerce').fillna(0)

# Group by author_association and compute mean comments
author_group = df.groupby('author_association')['comments'].agg(['count', 'mean', 'median']).reset_index()
author_group = author_group.sort_values('mean', ascending=False)

print("\n📊 Author Association vs. Comments (Descriptive Stats):")
print(author_group)

# Correlation between author_association (as ordinal) and comments
mapping = {'NONE':0, 'CONTRIBUTOR':1, 'OWNER':2, 'MEMBER':3, 'COLLABORATOR':4, 'MANNEQUIN':5}
df['author_num'] = df['author_association'].map(mapping).fillna(0)
corr, pval = spearmanr(df['author_num'], df['comments'])

print(f"\n Spearman Correlation (Author Association vs. Comments): {corr:.4f} (p={pval:.4f})")

def count_labels_safe(label_str):
    if pd.isna(label_str):
        return 0
    try:
        if label_str.strip().startswith('['):
            return len(ast.literal_eval(label_str))
        else:
            return len(label_str.split(',')) if ',' in label_str else 1
    except:
        return len(label_str.split(',')) if ',' in label_str else 1

df['label_count'] = df['labels'].apply(count_labels_safe)

# Correlation between label_count and comments
label_corr, label_pval = spearmanr(df['label_count'], df['comments'])

print(f"\n Spearman Correlation (Label Count vs. Comments): {label_corr:.4f} (p={label_pval:.4f})")

# Top labels analysis (optional)
from collections import Counter
import re

label_words = [word.strip().lower() for l in df['labels'].dropna() for word in re.split(r'[,|;]', l)]
label_freq = Counter(label_words)
print("\nTop 10 Labels by Frequency:")
print(label_freq.most_common(10))

/var/folders/x7/dd18rcl92fl22lg10gbjt8zm0000gn/T/ipykernel_29297/1107983517.py:6: DtypeWarning: Columns (6,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)



📊 Author Association vs. Comments (Descriptive Stats):
  author_association   count       mean  median
3          MANNEQUIN     213  10.765258     5.0
4             MEMBER   32920   6.425668     2.0
2        CONTRIBUTOR   97710   4.720612     2.0
5               NONE  204097   4.437277     2.0
1       COLLABORATOR   23708   4.022608     1.0
6              OWNER   45418   1.240411     0.0
0                1.0       1   0.000000     0.0

📈 Spearman Correlation (Author Association vs. Comments): -0.0867 (p=0.0000)

📈 Spearman Correlation (Label Count vs. Comments): 0.3510 (p=0.0000)

Top 10 Labels by Frequency:
[("'default': false", 493499), ('[]', 172114), ("'description': ''}", 89734), ("'description': none}", 79451), ("'default': true", 72096), ("'color': 'ededed'", 63168), ("'description': ''}]", 62442), ("'description': none}]", 56432), ("'name': 'bug'", 34753), ("'name': 'enhancement'", 23522)]


OWNER/MEMBER tend to receive fewer comments.might suggest that issues created by OWNERS/MEMBERS are clearer, easier to resolve, or receive less discussion.
Spearman Correlation: +0.351 shows a positive correlation which means that issues with more labels tend to have more comments.
Many lbels appear to be JSON fragments or API responses, not clean labels like accessibility, bug so I think if we need to get a better insignt on  labels we might need to clean this

Understand whether specific topics in the titles and body text are linked to higher engagement 
TF-IDF Vectorization on Titles + Body
LDA Topic Modeling



In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
from sklearn.preprocessing import LabelEncoder

# === Load Data ===
csv_path = '/Users/hanfuhou/Downloads/2004to2024all.csv' 
df = pd.read_csv(csv_path)

# === Combine Title + Body for Topic Modeling ===
df['text'] = df['title'].fillna('') + ' ' + df['body'].fillna('')

# === TF-IDF Vectorization ===
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
X_tfidf = tfidf.fit_transform(df['text'])

# === LDA Topic Modeling ===
lda = LatentDirichletAllocation(n_components=5, random_state=42)
topic_probs = lda.fit_transform(X_tfidf)

df['topic'] = np.argmax(topic_probs, axis=1)

# === Average Comments & Stars per Topic ===
df['comment_count'] = pd.to_numeric(df['comments'], errors='coerce').fillna(0)
df['stars'] = pd.to_numeric(df['score'], errors='coerce').fillna(0)

topic_summary = df.groupby('topic').agg({
    'comment_count': ['mean'],
    'stars': ['mean'],
    'author_association': lambda x: dict(Counter(x))
}).reset_index()

print(topic_summary)

# === Top Keywords per Topic ===
feature_names = tfidf.get_feature_names_out()
for idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[-10:][::-1]]
    print(f" Topic {idx} top words:", top_words)

/var/folders/x7/dd18rcl92fl22lg10gbjt8zm0000gn/T/ipykernel_29297/3310591718.py:10: DtypeWarning: Columns (6,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


  topic comment_count     stars  \
                 mean      mean   
0     0      7.100754  1.000000   
1     1      4.403297  1.000000   
2     2      3.590476  0.999938   
3     3      4.697235  1.000000   
4     4      0.006289  1.000000   

                                  author_association  
                                            <lambda>  
0  {'NONE': 36149, 'CONTRIBUTOR': 4201, 'OWNER': ...  
1  {'CONTRIBUTOR': 6789, 'NONE': 25882, 'OWNER': ...  
2  {'MEMBER': 3308, 'OWNER': 10000, 'NONE': 9831,...  
3  {'COLLABORATOR': 19497, 'OWNER': 19688, 'NONE'...  
4  {'NONE': 48453, 'COLLABORATOR': 9, 'CONTRIBUTO...  
 Topic 0 top words: ['talkback', 'version', 'android', 'google', 'com', 'issue', 'nextcloud', 'app', 'reported', 'original']
 Topic 1 top words: ['td', 'tr', 'java', 'react', 'eslint', 'javascript', 'mozilla', 'autojs', 'version', 'js']
 Topic 2 top words: ['td', 'https', 'tr', 'accessibility', 'com', 'github', 'alt', 'lighthouse', 'li', 'html']
 Topic 3 top words: [

I think for future if we are doing AI benchmarking Topics 0 & 3 could be strong candidates — highly engaged and user-facing.
Topic 4 Almost no comments. Might be ignored issues.

In [7]:
# Ensure types are ready
df['comment_count'] = pd.to_numeric(df['comments'], errors='coerce').fillna(0)
df['topic'] = df['topic'].fillna(-1)
df['author_association'] = df['author_association'].fillna("UNKNOWN")

# Group: average comments per author role per topic
pivot = df.groupby(['author_association', 'topic'])['comment_count'].mean().unstack().fillna(0)

# Show summary table
print("\n  Average Comment Count per Author Association × Topic:")
print(pivot)




  Average Comment Count per Author Association × Topic:
topic                      0         1          2          3         4
author_association                                                    
1.0                 0.000000  0.000000   0.000000   0.000000  0.000000
COLLABORATOR        5.800346  7.287454   2.236061   3.984254  2.222222
CONTRIBUTOR         7.273982  4.176020   6.473372   4.485045  3.857143
MANNEQUIN           8.380645  7.600000  80.000000  20.774194  0.000000
MEMBER              7.227349  6.727664   5.568924   6.461500  1.312500
NONE                7.473983  6.188625   3.792391   5.225147  0.004809
OWNER               2.708124  0.361858   1.080800   1.667005  0.307692
UNKNOWN             0.000000  0.000000   0.000000   0.000000  0.000000


OWNER issues usually have few comments - lower collaboration or clearer internal tasks
Topic 4 lower topics , maybe low priority or outdated topics
contributor and members have more comments (high engagement)
MANNEQUIN has extreme outlier in Topic 2 

Extract some example with topic 4 to see why less engagement 

In [9]:
df['comment_count'] = pd.to_numeric(df['comments'], errors='coerce').fillna(0)


topic_4_issues = df[df['topic'] == 4]

# Sort by comment count descending
top_topic_4_issues = topic_4_issues.sort_values(by='comment_count', ascending=False)

# Select relevant columns
top_topic_4_info = top_topic_4_issues[['title', 'author_association', 'comment_count']].head(30)

print(" Top 10 Most-Commented Issues in Topic 4:")
print(top_topic_4_info.to_string(index=False))

 Top 10 Most-Commented Issues in Topic 4:
                                                                                                    title author_association  comment_count
                                 Client doesn't start after downloading svg symbols: MAGE Client UI error               NONE           18.0
                                                                                       Osx 10.12.1 Sierra               NONE           13.0
                                   java.io.FileNotFoundException: ./interface (No such file or directory)               NONE           12.0
                                                                                    Pulse width traceback               NONE           11.0
                                                                                          Can't run 1.8.0        CONTRIBUTOR           10.0
                                                        Cannot add keywords with "Manage keywords" widget             

I think all of these from topic 4 are about java issue

 Issue Trends Over Time: Lifecycle & Resolution Patterns
 Extract created_at year for each issue.
	•	Count total issues created per year.
	•	Separate by:
	•	All issues
	•	Accessibility-labeled issues (filter using keywords like 'a11y', 'accessibility', 'screenreader', etc.).

    


In [12]:
import pandas as pd
import numpy as np


csv_path = '/Users/hanfuhou/Downloads/2004to2024all.csv' 
df = pd.read_csv(csv_path, low_memory=False)

# 1Extract created_year from created_at
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
df['created_year'] = df['created_at'].dt.year

# 2️Flag accessibility-labeled issues (in title or labels)
accessibility_keywords = ['a11y', 'accessibility', 'screenreader', 'aria', 'keyboard', 'alt text', 'alt-text','colorblind','blind','deaf','hearing','ally']
pattern = '|'.join(accessibility_keywords)

df['is_accessibility'] = (
    df['title'].fillna('').str.lower().str.contains(pattern) |
    df['labels'].fillna('').str.lower().str.contains(pattern)
)

# 3️⃣ Count total and accessibility issues per year
yearly_issues = df.groupby('created_year').size().rename('total_issues')
yearly_a11y_issues = df[df['is_accessibility']].groupby('created_year').size().rename('accessibility_issues')

yearly_counts = pd.concat([yearly_issues, yearly_a11y_issues], axis=1).fillna(0).astype(int)

# 4️⃣ Open vs. Closed issue status per year
issue_status_yearly = df.groupby(['created_year', 'state']).size().unstack(fill_value=0)
issue_status_yearly['total'] = issue_status_yearly.sum(axis=1)
issue_status_yearly['%_open'] = issue_status_yearly['open'] / issue_status_yearly['total'] * 100
issue_status_yearly['%_closed'] = issue_status_yearly['closed'] / issue_status_yearly['total'] * 100

# 5️⃣ Open vs. Closed for accessibility-labeled issues specifically
a11y_status_yearly = df[df['is_accessibility']].groupby(['created_year', 'state']).size().unstack(fill_value=0)
a11y_status_yearly['total'] = a11y_status_yearly.sum(axis=1)
a11y_status_yearly['%_open'] = a11y_status_yearly['open'] / a11y_status_yearly['total'] * 100
a11y_status_yearly['%_closed'] = a11y_status_yearly['closed'] / a11y_status_yearly['total'] * 100

# Display the results
print("\n Total & Accessibility Issues per Year:")
print(yearly_counts.tail(10))  # last 10 years for brevity

print("\n  Issue Status (Open vs Closed) by Year:")
print(issue_status_yearly.tail(10))

print("\n Accessibility Issues Status (Open vs Closed) by Year:")
print(a11y_status_yearly.tail(10))



 Total & Accessibility Issues per Year:
              total_issues  accessibility_issues
created_year                                    
2015.0               20146                  5668
2016.0               22268                  4223
2017.0               51197                  5825
2018.0               41012                  8270
2019.0               29381                 11272
2020.0               38856                 14338
2021.0               40909                 15043
2022.0               44295                 16249
2023.0               50401                 17629
2024.0               57704                 18279

  Issue Status (Open vs Closed) by Year:
state         closed   open  total     %_open   %_closed
created_year                                            
2015.0         13881   6265  20146  31.097985  68.902015
2016.0         19312   2956  22268  13.274654  86.725346
2017.0         42868   8329  51197  16.268531  83.731469
2018.0         23253  17759  41012  43.30196

In [14]:
active_repos_per_year = df.groupby('created_year')['repository_url'].nunique()
yearly_counts['normalized_total_issues'] = yearly_counts['total_issues'] / active_repos_per_year
yearly_counts['normalized_a11y_issues'] = yearly_counts['accessibility_issues'] / active_repos_per_year
print("\n Normalized Issue Counts by Year:")
print(yearly_counts)







 Normalized Issue Counts by Year:
              total_issues  accessibility_issues  normalized_total_issues  \
created_year                                                                
2004.0                  10                     5                 2.000000   
2005.0                  11                     1                 1.222222   
2006.0                  26                     9                 1.300000   
2007.0                  22                     5                 2.200000   
2008.0                  43                     9                 1.720000   
2009.0                  47                    10                 1.382353   
2010.0                 218                    59                 2.626506   
2011.0                 495                   121                 2.004049   
2012.0                 968                   262                 1.857965   
2013.0                2039                   644                 1.850272   
2014.0                4017               

Total issues have generally increased over time
Open accessibility issues highest in 2015 (37.5%), dropped in subsequent years, but started rising again since 2022.
By 2024 over 36% of accessibility issues remain open, maybe possible delays, complexity, or growing challenges specifically for accessibility.